# Problem 2: Warehouse optimization outline

Aaron Bell

# First thoughts

My first impression is that this is a bit different from typical optimization problems.

We want to optimize a continous value: cost, of delivery and distribution.
    
But the "parameters" we can work with are not necessarily all continuous. 
We can't physically adjust the location or size of the warehouses. Well, we **can** but this is usually wildly impractical. And until the age of floating warehouses comes, we're stuck with the locations we have.

Inside the warehouse we have some flexibility, but from the outside, we're limited to a set of discrete values. Locaiton and Size of Warehouse A, Location and Size of Warehouse B, and so on.

Thus we sort of have a 2-tiered problem: which warehouse do we put stuff an, and once it's there how to we deal with it.

The second part is probably the one that'll take us deeper down the rabbit hole. There's a lot of ways to organize stuff in a warehouse, a lot of different systems we can envision.

In the next section, I'll try to break the problem down into the data and parameters we have to work with.


\section{Parameters of warehouse selection}

These are attributes of each incoming product. Basically, I'm imagining this as a product-by-product decision making tree. For each new product we want to add to the inventory, we look at its inherent atrributes, and its relational attributes (things that depend on the current inventory, or on the customer.)

\subsection{Product Inherent attributes}

* **Product dimensions**
* **Product weight**
* **Flamability**

\subsection{Product Relational attributes}

* **Relatedness**

This one is really a list of attributes--- for each product we want to consider, it has a list of probabilities for each **other** product already in our inventory. These give the probabiltiy that a given product will be purchased along with a given other product.

* **Customer location**

Assuming our 'historical order data' includes customer locations, we can infer a probability distribution--- a map showing the most likely delivery areas for a given product. (Well, really it should be a probability distribution for each combination of products? A product by itself might have a different likely delivery zone from a pair of objects

\subsection{Warehouse inherent attributes}

* **Available space**
* **Does it have flammable stuff in it**
* **Location**

\subsection{Parameters we can control}

* **Which warehouse the thing goes to**


In other words, we want to choose the warehouse that maximizes overall product inter-relatedness, minimizes the overal "probability distance" that the products inside are likely to travel once purchased. 

For this problem, I'll consider that product relatedness and customer location, because these are things we have to estimate (predict). They're not definitive values that we can know or measure, unlike product dimensions.


\section{Warehouse as Blackhole}

If every warehouse had an infinite capacity, and every product we threw at it could be thought of as being essentially at the same point, we'd have a black hole warehouse. The warehouse with the greatest "gravitational pull" (optimal number of related products inside, and distance from likely delivery site), is the one that gets the product.

In this case, we're paring the problem down to simply a warehouse selection problem. Since warehouses don't move (yet?), we are just maximizing the "relatedness" of products---keeping those likely to be purchased together in the same warehouse. We have to keep in mind though that organization of products **inside** the warehouse is not trivial.

We'll refer to relatedness as $r$, and consider it to be a property of $product_{A}$, given an existing inventory. In other words, $r$ depends on the product in question, and the warehouse we intend to put it in.

So a product goes to whichever warehouse has the most "related items" in it, or the highest number of items that are frequently bought with it. But what if **Warehouse A** is in Chiba, but the area with the highest demand for the product is in central Tokyo? Perhaps you want to put the product in a more convenient warehouse. Or at least you want to consider which warehouse minimizes the total travel time, to the customers doorstep. We haven't yet optimized anyproduct but the likliness that two products in a warehouse will be bought together, let alone care about seasonal variation of purchase patterns, or flamability.

We can play with models, math and pseudo-code until our imaginations run dry, but another approach is experiment. If have access to seasonal purchase data, and knowledge of the door-to-door travel time for our products, we can test the effects of different distribution strategies.

\section{This is a section}
In an adaptive, data-driven approach, we can setup a running optimization problem. Whatever optimization algorithm we used, be it simple regression, or a time-series neural network (long-short term memory, etc.), we continually train our model to new data. 

What we'll need to estimate, is the relational attributes of each product, for each possible choice of warehouse.   

\subsection{Product-relatedness}

The binary "related or not related" score from above is a bit silly. 

What we will deal with instead, are continuous probabilities for each product-pair. In reality, there should be a probability $P(product_{A}|product_{i})$, for any given $product_{A}$, that it will be purchased along with any of the other $product_{i}$s in the inventory-- the % chance that $product_A$ will be carted with $product_B$. 

Getting these probabilities will probably be harder than the actual warehouse-optimization, once we have them. We need to have not only a long history of product purchase data, to be able to predict co-purchase probability for a particular season, but we also have to have enough resolution in our data to make predictions down to a given week or day (as needed). 

Then we do something like, ask how many of the total times $product_A$ was purchased, was $product_B$ also purchased? That fraction would give us the co-purchase probability for the pair of products. It sounds easy enough, but we'd at least have to know this pair-probability for our entire inventory. Ideally, for the whole catalog of products we deal with.

And we would have to make this prediction down to some sufficiently short timescale.. per day? We can do something crude, calculate probabilities for March 8th 2018 based on purchase data for March 8th of previous years, but that doesn't help us for new products. It also leaves us blind to shorter scale patterns (day of the week), or one-off events. 

We'll need a method take takes into account both short-term and long-term fluctuations. While I'm not an expert in any of the following, these are some of the methods I would explore for predicting product relatedness $r$ based on a mix of long and short term data

* **Gaussian process modeling**
   *  There is a nice example by astronomer Anna Scalfe, using Guassian process modeling to predict CO2 levels based on long and short term data https://events.asiaa.sinica.edu.tw/school/20170904/talk/scaife2.pdf)}

* **Autoregressive integrated moving average model** (ARIMA)
   *  ARIMA can be implemented fairly painlessly with the 'statsmodels' python package: http://www.statsmodels.org/stable/examples/notebooks/generated/tsa_arma_0.html
   * A nice overview can be found here:   https://www.quora.com/What-is-ARIMA
   
And then the relatedness will probably need to be convolved with some sort of spatial probability model of where a given product or product combination is liklely to be sold. Ideally, they should be warehoused close to the final delivery point


\section{Optimization problem}

Assuming we're able to make good predicitons of co-purchase probability, likely delivery zones, etc., we're still left with the warehouse choice itself!

If all the warehouses were in the same place, and the customer location function were the same (all the likely customers happen to live in the same location), we'd just choose the one that maximizes product relatedness.

If they all had the same product relatedness, we'd just choose the un-full warehouse closest to the customer pool.

But in reality, none of these things are pinned-down so easily. Our optmization problem has to give each warehouse a score, taking into account product relations, customer locations, flamability, available space in a warehouse, etc. But which of these parameters is more important isn't obvious--- we have to find the "weights" telling us how these different parameters should be scored relative to each other, and a change in any one of them affects the overall cost. 

The only parameter that has an obvious degree of importance is flamability. Under no circumstances can we put a flammable object in a warehouse holding inflammable objects, no matter how optimal that warehouse seems. So we'll have to write our algorithm such that flammability of the object has an absolute priority. All of the other factors must be balanced against each other.

This is a situation where we may be able to fit the cost-per-parameter. Given past data, we can try to fit weights based on the known cost per order-- the total distance the order had to travel, and time it took to pick and ship it. This would require us to know the product attributes at the time the product was purchased.


In the end, we would want to maximuze a function that may look something like the following:

$Score =  f(A(r) - B(d))$

where $f$ is a binary "flammability coefficient". It takes a value of zero if a given products flammability doesn't match that of the warehouse being tested, and 1 otherwise. $r$ is the relatedness score between a given a product, and the present inventory in the warehouse. $d$ is a distance penalty score, an estimate how far a given product, and it's most common pairings, are likely to travel before delivery.

$A$ and $B$ are weights, to be determined by experimentation or perhaps even a more sophisticated approach like neural networks. These may need to be constantly adjusted as new data and products are introduced.

The warehouse with the highest score is the one that we place the product in.

\section{Comment: Inside the warehouse}

Everything discussed above essentially ignores product sorting **inside** the warehouse. I've just assumed that these steps warehouse choice, can be isolated from inner-warehouse processing. I.e. But of course we may also want to put $product_A$ close to $product_B$ **inside** the warehouse. That way they can be "picked" with the minimum travel time. 

But realistically, you couldn't assign  to **Warehouse 1** unless that warehouse had an available **Slot X**. Otherwise you end up blindly throwing stuff at a warehouse is already overcapacity (Black-holes don't actually make good warehouses.) So at the very least, these two steps: **Warehouse Selection** and **Warehouse Organization** need to be coupled. At the very least, there needs to be a feedback loop that disqualifies a warehouse if there's little or no space left. Ideally though, our algorithm should weigh the estimated processing time inside each warehouse, against what happens on the outside.

This is where our product dimensions and mass will become more useful.

In the interest of time however, I haven't gone into the optimization of product arrangement in the warehouse. Frankly, I spent too much time thinking through the warehouse selection steps. So this part I'll consider as a future excercise, though I did find at least one interesting reference on the topic:

* **An Overview of Warehouse Optimization** - Jan Karasek
https://www.researchgate.net/publication/260742754_An_Overview_of_Warehouse_Optimization
(Note that this is more about the overall layout of a warehouse, rather than optimizing co-purchase processing time.)



\section{Appendix}

Simple Pseudo-code implementation of warehouse choice, assuming product pairs take binary values for their relatedness score. A given $proudct_A$ is either likely (1) or not likely (0) to be purchased along with $product_B$.

def getInventoryAt(warehouse):

    Things = [query the warehouse]
    
    return Things
    
    
   
def testRelatedness(product_a, product_b):
    
    
    if isRelated(product_a
    
    
    
def relatedness(product, warehouse):

    Things = getInventoryAt(warehouse)
    
    relatedness_score = 0
    
    for other_product in Things:
        
        if isRelated(product, other_product) == True:
            relatedness_score += 1
    
    return relatedness_score
    
def findOptimalWarehouse(Warehouses):

    warehouse_scores = []

    for warehouse in Warehouses:
        
        score = relatedness(product, warehouse)
        
        warehouse_scores.append(score)
     
    optimal_score = np.max(warehouse_scores)
    
    optimal_warehouse = Warehouses[warehouse_scores == optimal_score]
     
    
```
